In [47]:
import requests
import json
from datetime import datetime

BASE_URL = "https://clinicaltrials.gov/api/v2/studies"

def fetch_full_trial(query_term="diabetes"):
    url = f"{BASE_URL}?query.term={query_term}&pageSize=5"
    print("🔍 正在抓取完整試驗資料...")

    response = requests.get(url)
    if response.status_code != 200:
        print(f"❌ 錯誤：{response.status_code} - {response.text}")
        return []

    data = response.json()
    studies = data.get("studies", [])
    if not studies:
        print("⚠️ 沒有資料")
        return []

    return studies

def save_to_json(data, filename):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✅ 已儲存到 {filename}")

# 攤平函式
def flatten_dict(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            for i, item in enumerate(v):
                if isinstance(item, dict):
                    items.extend(flatten_dict(item, f"{new_key}[{i}]", sep=sep).items())
                else:
                    items.append((f"{new_key}[{i}]", item))
        else:
            items.append((new_key, v))
    return dict(items)

if __name__ == "__main__":
    trials = fetch_full_trial(query_term="diabetes")
    
    if trials:
        today_str = datetime.today().strftime("%Y%m%d")
        save_to_json(trials, f"full_trials_{today_str}.json")

        for trial in trials:
            flat_trial = flatten_dict(trial)
            
            # 以 nctID + 抓取資料日期命名
            nct_id = flat_trial.get("protocolSection.identificationModule.nctId", "unknown")
            
            flat_filename = f"{nct_id}_{today_str}.json"
            save_to_json(flat_trial, flat_filename)

🔍 正在抓取完整試驗資料...
✅ 已儲存到 full_trials_20250702.json
✅ 已儲存到 NCT02536248_20250702.json
✅ 已儲存到 NCT06047548_20250702.json
✅ 已儲存到 NCT00796848_20250702.json
✅ 已儲存到 NCT00846248_20250702.json
✅ 已儲存到 NCT06977048_20250702.json
